### Example code to extract MGL chapter-sections pairs from the bills, and scraping the MGL sections based on the chapter-section pairs using the API. 
The output of the function used is pandas dataframe containing added columns 
 - Chapter-section pairs extracted
 - List containing MGL sections text
 - MGL sections combined into one string
 - Chapter-section pairs that returned no text (no sections were found for the given pair - likely because they are the new sections being added)
 - Word count for the MGL sections for a given bill

In [1]:
import extract_mgl_sections as ex
import pandas as pd

In [2]:
bills_csv_file_path = "12billswithmgl.csv" 
bill_title_col_name = "Title"
bill_num_col_name = "BillNumber"
docket_num_col_name = "DocketNumber"
bill_text_col_name = "DocumentText" 
chap_sec_lists_col_name = "Chapter-Section List"
word_count_col_name = "WordCount"

#kept the 'Combined_MGL' column name hard coded in the original function as it is used later in the demo_app.py and app2.py

mgl_df = ex.get_df_with_mgl(bills_csv_file_path, bill_title_col_name, bill_num_col_name, docket_num_col_name, bill_text_col_name, chap_sec_lists_col_name, word_count_col_name)

Progress: 100.00%

In [3]:
mgl_df

,Bill Info,Title,BillNumber,DocketNumber,DocumentText,Chapter-Section List,MGL Sections List,Combined_MGL,No MGL Found,WordCount
0,H3069: An Act relative to collective bargainin...,An Act relative to collective bargaining right...,H3069,HD2435,\tSECTION 1. Section 1 of Chapter 150E of the ...,"[[150E, 1], [150E, 3], [268A, 23]]",[Section 1. The following words and phrases as...,Section 1. The following words and phrases as ...,[],5255
1,H3121: An Act relative to the open meeting law,An Act relative to the open meeting law,H3121,HD2204,\tSECTION 18 of chapter 30A of the General Law...,"[[30A, 18]]",[Section 18. As used in this section and secti...,Section 18. As used in this section and sectio...,[],626
2,H400: An Act relative to vehicle recalls,An Act relative to vehicle recalls,H400,HD1240,"\tSECTION 1. Chapter 90 of the General Laws, a...","[[90, 7A], [90, 30119], [90, 30120]]",[Section 7A. The registrar shall establish rul...,Section 7A. The registrar shall establish rule...,"[[90, 30119], [90, 30120]]",1313
3,H489: An Act providing affordable and accessib...,An Act providing affordable and accessible hig...,H489,HD2794,\tSECTION 1. The seventh paragraph of section ...,"[[7, 22N], [15D, 1A], [15D, 2], [15D, 5], [15D...",[Section 22N. The operational services divisio...,Section 22N. The operational services division...,"[[15D, 13A], [15D, 19]]",107140
4,"H538: An Act to Improve access, opportunity, a...","An Act to Improve access, opportunity, and cap...",H538,HD485,\tSECTION 1. To create a new capital infrastru...,"[[70, 1A], [70B, 10], [59, 21C], [69, 37], [69...",[Section 10. The grant percentage for approved...,Section 10. The grant percentage for approved ...,"[[70, 1A], [69, 37], [69, 38], [70, 10A]]",15250
5,H711: An Act providing a local option for rank...,An Act providing a local option for ranked cho...,H711,HD1183,\tSECTION 1. Section 44A of chapter 43 of the ...,"[[43, 44A], [54, 103R], [54, 77], [54, 103Q]]","[Section 44A. In every city, governed on Septe...","Section 44A. In every city, governed on Septem...","[[54, 103R]]",1109
6,S1979: An Act establishing a jail and prison c...,An Act establishing a jail and prison construc...,S1979,SD661,\tSECTION 1. Chapter 7C of the General Laws is...,"[[7C, 72], [7C, 73], [125, 1], [7C, 2]]","[Section 72. The commissioner shall, each year...","Section 72. The commissioner shall, each year,...","[[7C, 73]]",1678
7,S2014: An Act relative to collective bargainin...,An Act relative to collective bargaining right...,S2014,SD1208,\tSECTION 1. Section 1 of chapter 150E of the ...,"[[150E, 1], [150E, 3], [268A, 23]]",[Section 1. The following words and phrases as...,Section 1. The following words and phrases as ...,[],5255
8,S2064: An Act extending the public records law...,An Act extending the public records law to the...,S2064,SD390,\tSECTION 1. Section 7 of chapter 4 of the Gen...,"[[4, 7], [66, 1A], [66, 6A], [66, 10], [66, 18]]",[Section 7. In construing statutes the followi...,Section 7. In construing statutes the followin...,[],31962
9,S257: An Act to end discriminatory outcomes in...,An Act to end discriminatory outcomes in vocat...,S257,SD2312,\tSECTION 1. Chapter 74 of the General Laws is...,"[[74, 5B], [74, 5C]]",[Section 5B. Regional school districts or citi...,Section 5B. Regional school districts or citie...,"[[74, 5C]]",44
